# Zion Data Analysis Notebook

Defs, Imports, etc

In [ ]:
import os, sys
from glob import glob
import time
from collections import UserDict, UserString
import numpy as np
import pandas as pd
from skimage.color import rgb2hsv
from skimage import filters, morphology, segmentation, measure
from scipy.optimize import nnls
from tifffile import imread, imwrite
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [11, 8]

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(os.path.join(module_path))
    

from image_processing.ZionImage import ZionImage, create_labeled_rois, get_wavelength_from_filename, get_cycle_from_filename, get_time_from_filename, get_imageset_from_cycle
from image_processing.ZionBase import df_cols, extract_spot_data, csv_to_data, crosstalk_correct, display_signals, create_phase_correct_matrix, base_call, add_basecall_result_to_dataframe


### Give the notebook a directory of tiffs to work in and some parameters:

In [ ]:
input_dir_path = "C:\\Users\\ZacharyAugenfeld\\Documents\\ZionData\\raws"

# This parameter determines whether temporal difference images are used
useDifferenceImage = False

# This is the name of the UV channel
uv_wl = "365"

# This determines what type of background subtraction is used
bgSubtract = False

useTiff = False

# This was used to help with file identification
#bTiff = False


## Step 1: Spot Detection

### Define parameters used in spot detection. 

When you run this cell, ROIs will be displayed. Verify you get the spots you want. (If not, adjust parameters here and re-run.)

In [ ]:
# This determines size of noise filter kernel (prob doesn't need to be changed much)
median_kernel_size = 9

# These tune spot size and how much spot border is included
erode_kernel_size = 17
dilate_kernel_size = 13

# This adjusts sensitivity for thresholding (should be close to 1)
threshold_scale = 0.25

# These are constraints on how large or small a spot could be
spotMinSize = 1200
spotMaxSize = None

# This is a list of RGB weights when turning the (UV) image to grayscale
rgb_weights = (0.4, 0.2, 0.1)
#rgb_weights = None

'''If roi_label_imagefile is set to None, spot detection will be performed.
   If it is set to a file, it will load spot ROIs and NOT run spot detection,
   and the above spot detection parameters will be ignored.'''

roi_label_imagefile = None
#roi_label_imagefile = "roi_map.tif" #useful if already run



#### DO NOT EDIT BELOW THIS LINE ####
cycle1ImageSet = get_imageset_from_cycle(1, input_dir_path, uv_wl, useDifferenceImage, useTiff=useTiff)

if roi_label_imagefile is not None:
    spot_labels = imread(os.path.join(input_dir_path, roi_label_imagefile))
else:
    _,spot_labels,_ = cycle1ImageSet.detect_rois( os.path.join(input_dir_path), median_ks=median_kernel_size, erode_ks=erode_kernel_size, dilate_ks=dilate_kernel_size, threshold_scale=threshold_scale, minSize=spotMinSize, maxSize=spotMaxSize, gray_weights=rgb_weights)

rois_img = create_labeled_rois(spot_labels)
imwrite(os.path.join(input_dir_path, "rois_img.tif"), rois_img)
plt.imshow(rois_img)

#TODO add other excitation images with ROI overlay

## Step 2: Define color dictionary

Either provide a file OR select spot (by label) for each base:

In [ ]:
# For each base, provide purest spot (by label in image above): 
color_A_spot = 3
color_C_spot = 19
color_G_spot = 14
color_T_spot = 20

'''If basis_colors_file is set to None, spot detection will be performed.
   If it is set to a file, it will load color dictionary from file,
   ignoring the above basis color spots.'''

basis_colors_file = None
#basis_colors_file = os.path.join(input_dir_path, "M.npy")

# Define what spot is used as background/reference spot
# Note that this is only used if bgSubtract above is True
bg_spot = None

#### DO NOT EDIT BELOW THIS LINE ####
if basis_colors_file is not None:
    M = np.load(basis_colors_file)
else:
    M = np.zeros(shape=(3*(cycle1ImageSet.nChannels-1), 4))
    nSpots = np.max(spot_labels)
    for spot in range(1,nSpots+1):
        for base_spot_ind, base_spot in enumerate([color_A_spot, color_C_spot, color_G_spot, color_T_spot]):
            if spot==base_spot:
                #print(f"setting base index (column) {base_spot_ind} to mean at spot {spot}")
                vec = cycle1ImageSet.get_mean_spot_vector( spot_labels==spot )
                #TODO: should we normalize anything here?
                M[:,base_spot_ind] = vec
    np.save(os.path.join(input_dir_path, "M.npy"), M)
print(f"M = \n{M}")

## Step 3: Extract spot data and write to CSV

In [ ]:
numCycles = 7

csvfile = os.path.join(input_dir_path, "basecaller_spot_data.csv")
with open(csvfile, "w") as f:
    f.write(','.join(df_cols)+'\n')
    
for new_cycle in range(1,numCycles+1):
    currImageSet = get_imageset_from_cycle(new_cycle, input_dir_path, uv_wl, useDifferenceImage, useTiff=useTiff)
    spot_data = extract_spot_data(currImageSet, spot_labels, csvFileName=csvfile)

## Step 4: Perform base-calling

### First, determine signal pre-phase-correction:
Define number of cycles COLLECTED (phase-corrected output of base-caller will be 1 fewer):

In [ ]:
#### DO NOT EDIT BELOW THIS LINE ####
# todo kinetics figure, similar to below
# generate pre-phase-correction histograms:
basecall_csv = csvfile

#todo: add option to use median instead of mean

basecall_pd = csv_to_data(basecall_csv)
signal_pre_basecall, spotlist, basecall_pd_pre = crosstalk_correct(basecall_pd, M, numCycles)
#spotlist = sorted(spotlist)
basecall_pd_pre.to_csv(os.path.join(input_dir_path, "basecaller_output_data_pre.csv"))

### Next, define p and q for phase correction matrix:

In [ ]:
p = 0.015 # Probability that ZERO new base synthesized during a cycle (lagging)
q = 0.1   # Probability that TWO new bases synthesized during a cycle (leading)

#### DO NOT EDIT BELOW THIS LINE ####
Qinv = create_phase_correct_matrix(p,q,numCycles)
signal_post_basecall = np.transpose( (np.transpose(signal_pre_basecall, axes=(0,2,1)) @ Qinv)[:,:,:-1], axes=(0,2,1))
basecall_pd_post = add_basecall_result_to_dataframe(signal_post_basecall, basecall_pd)
basecall_pd_post.to_csv(os.path.join(input_dir_path, "basecaller_output_data_post.csv"))


### Now display results and write to report
Define report filename and number of rows in figure outputs.

In [ ]:
# You can name report file here:
reportfile = os.path.join(input_dir_path, "report.txt")

figNumRows = 5

#TODO: spot name lookup table

#### DO NOT EDIT BELOW THIS LINE ####
f1, f2 = display_signals(signal_pre_basecall, spotlist, numCycles, numRows=figNumRows, preOrPost="pre")
f3, f4 = display_signals(signal_post_basecall, spotlist, numCycles-1, numRows=figNumRows, preOrPost="post")

for f_idx, f in enumerate(f1):
    f.savefig(os.path.join(input_dir_path, f"Purity Pre-Phase {f_idx+1}.png"))
for f_idx, f in enumerate(f2):
    f.savefig(os.path.join(input_dir_path, f"Signal Pre-Phase {f_idx+1}.png"))
for f_idx, f in enumerate(f3):
    f.savefig(os.path.join(input_dir_path, f"Purity Post-Phase {f_idx+1}.png"))
#for f_idx, f in enumerate(f4):
#    f.savefig(os.path.join(input_dir_path, f"Signal Post-Phase {f_idx+1}.png"))

#TODO add more metrics to report
#TODO package up as pdf?    

with open(reportfile, 'w') as f:
    if useDifferenceImage:
        print(f"Difference = Temporal", file=f)
    else:
        print(f"Difference = Dark", file=f)

    print(f"Median Filter Kernel Size = {median_kernel_size}", file=f)
    print(f"Erosion Kernel Size = {erode_kernel_size}", file=f)
    print(f"Dilation Kernel Size = {dilate_kernel_size}", file=f)
    print(f"Mean Threshold Scale Factor = {threshold_scale}", file=f)
    print(f"ROI labels at {os.path.join(input_dir_path, 'rois.jpg')}", file=f)
    print(f"'Cross-talk' matrix M = {M}", file=f)
    #todo list where output csv is?
    print(f"Pre-phase corrected Purity at {os.path.join(input_dir_path, 'Purity Pre-Phase.png')}", file=f)
    print(f"Pre-phase corrected Signal {os.path.join(input_dir_path, 'Signal Pre-Phase.png')}", file=f)
    print(f"Base-caller p = {p}", file=f)
    print(f"Base-caller q = {q}", file=f)
    print(f"Post-phase corrected Purity at {os.path.join(input_dir_path, 'Purity Post-Phase.png')}", file=f)
    print(f"Post-phase corrected Signal {os.path.join(input_dir_path, 'Signal Post-Phase.png')}", file=f)